In [ ]:
def collect_mlflow_model_perfs():
    import mlflow
    from mlflow.tracking import MlflowClient
    import os
    import pandas as pd
    import numpy as np
    
    # Connexion au serveur MLFlow
    mlflow.set_tracking_uri("http://localhost:5000")
    client = MlflowClient("http://127.0.0.1:5000")
    
    try:
        experiments = client.search_experiments(view_type=mlflow.entities.ViewType.ALL)
        #print("Connexion au serveur MLflow. Expériences détectées:")
        for exp in experiments:
            print(f"Expérience ID: {exp.experiment_id}, Nom: {exp.name}")
    except Exception as e:
        print("Échec de la connexion au serveur MLflow:", e)
    
    # Utiliser l'experiment par défaut avec ID = 0
    experiment_id = "0"
    
    # Récupérer les runs pour l'experiment par défaut
    runs = client.search_runs(experiment_ids=[experiment_id])
    
    # Afficher les noms des runs
    run_names = [run.data.tags.get("mlflow.runName") for run in runs]
    for idx, run_name in enumerate(run_names):
        #print(f"Run {idx+1}: {run_name}")
        None
    
    # Pour chaque run, lire les métriques loggées avec arrondi à 2 chiffres
    Runs=[]
    MetricName=[]
    MetricValue=[]
    for run in runs:
        #print(f"Run ID: {run.info.run_id}")
        #print(f"Run Name: {run.data.tags.get('mlflow.runName')}")
        
        # Afficher toutes les métriques loggées pour ce run
        metrics = run.data.metrics
        for metric_name, metric_value in metrics.items():
            #print(f"{metric_name}: {round(metric_value, 2)}")
            Runs.append(run.data.tags.get('mlflow.runName'))
            MetricName.append(metric_name)
            MetricValue.append(metric_value)
        
        #print("-" * 40)
    
    
    
    compilation = pd.DataFrame({"Runs":Runs,"MetricName":MetricName,"MetricValue":MetricValue})
    best_model_per_metric= compilation.groupby("MetricName").apply(lambda x: x[["Runs","MetricValue"]][x["MetricValue"]==np.max(x["MetricValue"])] )
    best_model_per_metric["MetricName"]=[i[0] for i in best_model_per_metric.index]
    #best_model_per_metric_test=best_model_per_metric.loc[best_model_per_metric.MetricName.str.contains("test"),:]
    #best_model_per_metric_test.sort_values("Runs")
    mlflow.end_run()
    return best_model_per_metric #best_model_per_metric_test
    
    


def collect_mlflow_model_hyperparams():
    import mlflow
    from mlflow.tracking import MlflowClient
    import pandas as pd

    # Step 1: Connect to MLflow server
    mlflow.set_tracking_uri("http://localhost:5000")
    client = MlflowClient("http://127.0.0.1:5000")

    try:
        # Step 2: Search for all experiments
        experiments = client.search_experiments(view_type=mlflow.entities.ViewType.ALL)
        print("Connexion au serveur MLflow. Expériences détectées:")
        for exp in experiments:
            print(f"Expérience ID: {exp.experiment_id}, Nom: {exp.name}")
    except Exception as e:
        print("Échec de la connexion au serveur MLflow:", e)
        return

    # Step 3: Use the default experiment (ID = 0)
    experiment_id = "0"

    # Step 4: Search for runs in the selected experiment
    runs = client.search_runs(experiment_ids=[experiment_id])

    # Step 5: Initialize lists to store the hyperparameters (params) for each run
    Runs = []
    ParamName = []
    ParamValue = []

    # Step 6: Loop through each run and collect hyperparameters
    for run in runs:
        print(f"Run ID: {run.info.run_id}")
        print(f"Run Name: {run.data.tags.get('mlflow.runName')}")

        # Retrieve all logged hyperparameters (params) for the current run
        params = run.data.params
        for param_name, param_value in params.items():
            print(f"Hyperparameter - {param_name}: {param_value}")
            Runs.append(run.data.tags.get('mlflow.runName'))
            ParamName.append(param_name)
            ParamValue.append(param_value)
        
        print("-" * 40)

    # Step 7: Create a DataFrame with the collected hyperparameters
    hyperparams_df = pd.DataFrame({"Run": Runs, "ParamName": ParamName, "ParamValue": ParamValue})
    
    # Optional: Sort or filter based on hyperparameters, if needed
    sorted_hyperparams_df = hyperparams_df.sort_values(by=["Run", "ParamName"])

    print("Hyperparameters collected:")
    #print(sorted_hyperparams_df)
    mlflow.end_run()

    return sorted_hyperparams_df


In [3]:
df=collect_mlflow_model_perfs()
df=df.reset_index(drop=True)
df.sort_values(['Runs','MetricName'])

Expérience ID: 0, Nom: Default


/tmp/ipykernel_224582/1909827535.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_model_per_metric= compilation.groupby("MetricName").apply(lambda x: x[["Runs","MetricValue"]][x["MetricValue"]==np.max(x["MetricValue"])] )


,Runs,MetricValue,MetricName
0,LogisticRegression_F10,0.629123,cv_test_mean_score_fbeta
1,LogisticRegression_F10,0.634312,cv_train_mean_score_fbeta
3,LogisticRegression_F10,0.650386,test_fbeta_score
6,LogisticRegression_F10,0.634181,train_fbeta_score
2,LogisticRegression_ROC_AUC,0.734182,test_cv_mean
4,LogisticRegression_ROC_AUC,0.687327,test_roc_auc
5,LogisticRegression_ROC_AUC,0.743167,train_cv_mean
7,LogisticRegression_ROC_AUC,0.742221,train_roc_auc
